In [1]:
import math
import os
import time
from warnings import filterwarnings

import pandas as pd
import pyodbc
from dotenv import load_dotenv
from tqdm import tqdm

filterwarnings("ignore")

In [ ]:
def conectar_sqlserver() -> pyodbc.Connection:
    """
    Cria conexão com SQL Server

    :return: Conexão pyodbc
    :rtype: pyodbc.Connection
    """
    load_dotenv()
    conn = pyodbc.connect(os.getenv("SQL_CONNECTION_STRING"))
    return conn


conn = conectar_sqlserver()

In [ ]:
def carregar_dados_superior_ies(arquivo_csv: str, dtype=str) -> pd.DataFrame:
    """
    Carrega e trata os dados do CSV de instituições de ensino superior.

    :param arquivo_csv: Caminho para o arquivo CSV
    :type arquivo_csv: str
    :param dtype: Tipo de dado para as colunas
    :return: DataFrame com os dados tratados
    :rtype: pd.DataFrame
    """
    df = pd.read_csv(
        arquivo_csv,
        sep=";",
        encoding="latin1",
        dtype=dtype,  # type: ignore
        low_memory=False,
    )

    # Tratamento de valores faltantes e padronização de colunas específicas
    categoria_desconhecido = "Desconhecido"

    df["NU_CEP_IES"] = df["NU_CEP_IES"].fillna(0).astype(str).str.zfill(8)
    df["SG_IES"] = df["SG_IES"].fillna(categoria_desconhecido).astype(str)
    df["DS_NUMERO_ENDERECO_IES"] = [
        i if "s/n" not in i.lower() and "sn" not in i.lower() and i != "-" else "0"
        for i in df["DS_NUMERO_ENDERECO_IES"].fillna("0").astype(str)
    ]
    df["DS_COMPLEMENTO_ENDERECO_IES"] = [
        i if "s/n" not in i.lower() and i != "-" else categoria_desconhecido
        for i in df["DS_COMPLEMENTO_ENDERECO_IES"]
        .fillna(categoria_desconhecido)
        .astype(str)
    ]
    df["TP_ORGANIZACAO_ACADEMICA"] = df["TP_ORGANIZACAO_ACADEMICA"].map(
        {
            "1": "Universidade",
            "2": "Centro Universitário",
            "3": "Faculdade",
            "4": "Instituto Federal de Educação, Ciência e Tecnologia",
            "5": "Centro Federal de Educação Tecnológica",
        }
    )
    if "TP_REDE" in df.columns:
        df["TP_REDE"] = df["TP_REDE"].map({"1": "Pública", "2": "Privada"}).astype(str)
    df["TP_CATEGORIA_ADMINISTRATIVA"] = df["TP_CATEGORIA_ADMINISTRATIVA"].map(
        {
            "1": "Pública Federal",
            "2": "Pública Estadual",
            "3": "Pública Municipal",
            "4": "Privada com fins lucrativos",
            "5": "Privada sem fins lucrativos",
            "6": "Privada - Particular em sentido estrito",
            "7": "Especial",
            "8": "Privada comunitária",
            "9": "Privada confessional",
        }
    )

    return df

In [ ]:
def carregar_dados_superior_curso(arquivo_csv: str) -> pd.DataFrame:
    """
    Carrega e trata os dados do CSV de cursos de ensino superior.

    :param arquivo_csv: Caminho para o arquivo CSV
    :type arquivo_csv: str
    :return: DataFrame com os dados tratados
    :rtype: pd.DataFrame
    """
    df = pd.read_csv(
        arquivo_csv,
        sep=";",
        encoding="latin1",
        dtype=str,
        low_memory=False,
    )

    # Tratamento de valores faltantes e padronização de colunas específicas
    categoria_desconhecido = "Desconhecido"

    df["NO_REGIAO"] = df["NO_REGIAO"].fillna(categoria_desconhecido).astype(str)
    df["CO_REGIAO"] = df["CO_REGIAO"].fillna(0).astype(int)
    df["NO_UF"] = df["NO_UF"].fillna(categoria_desconhecido).astype(str)
    df["SG_UF"] = df["SG_UF"].fillna(categoria_desconhecido).astype(str)
    df["CO_UF"] = df["CO_UF"].fillna(0).astype(int)
    df["NO_MUNICIPIO"] = df["NO_MUNICIPIO"].fillna(categoria_desconhecido).astype(str)
    df["CO_MUNICIPIO"] = df["CO_MUNICIPIO"].fillna(0).astype(int)

    df["IN_CAPITAL"] = [
        i if i != "." else "0" for i in df["IN_CAPITAL"].fillna("0").astype(str)
    ]

    df["TP_GRAU_ACADEMICO"] = (
        df["TP_GRAU_ACADEMICO"].fillna(categoria_desconhecido).astype(str)
    )

    df["TP_GRAU_ACADEMICO"] = df["TP_GRAU_ACADEMICO"].replace(
        {
            "1": "Bacharelado",
            "2": "Licenciatura",
            "3": "Tecnológico",
            "4": "Bacharelado e Licenciatura",
            "5": "Não aplicável",
        }
    )

    df["TP_DIMENSAO"] = df["TP_DIMENSAO"].map(
        {
            "1": "Cursos presenciais ofertados no Brasil",
            "2": "Cursos a distância ofertados no Brasil",
            "3": "Cursos a distância com dimensão de dados somente a nível Brasil",
            "4": "Cursos a distância ofertados por instituições brasileiras no exterior",
        }
    )
    df["TP_ORGANIZACAO_ACADEMICA"] = df["TP_ORGANIZACAO_ACADEMICA"].map(
        {
            "1": "Universidade",
            "2": "Centro Universitário",
            "3": "Faculdade",
            "4": "Instituto Federal de Educação, Ciência e Tecnologia",
            "5": "Centro Federal de Educação Tecnológica",
        }
    )

    df["TP_REDE"] = df["TP_REDE"].map({"1": "Pública", "2": "Privada"}).astype(str)
    df["TP_CATEGORIA_ADMINISTRATIVA"] = df["TP_CATEGORIA_ADMINISTRATIVA"].map(
        {
            "1": "Pública Federal",
            "2": "Pública Estadual",
            "3": "Pública Municipal",
            "4": "Privada com fins lucrativos",
            "5": "Privada sem fins lucrativos",
            "6": "Privada - Particular em sentido estrito",
            "7": "Especial",
            "8": "Privada comunitária",
            "9": "Privada confessional",
        }
    )
    df["TP_MODALIDADE_ENSINO"] = df["TP_MODALIDADE_ENSINO"].map(
        {"1": "Presencial", "2": "Curso a distância"}
    )
    df["TP_NIVEL_ACADEMICO"] = df["TP_NIVEL_ACADEMICO"].map(
        {"1": "Graduação", "2": "Sequencial de Formação Específica"}
    )

    return df

In [ ]:
def tratar_chaves_estrangeiras(
    foreign_keys: dict, df_filtrado: pd.DataFrame, cursor: pyodbc.Cursor, esquema: str
) -> None:
    """
    Verifica e mapeia chaves estrangeiras no DataFrame antes da inserção.

    :param foreign_keys: Dicionário de chaves estrangeiras
    :type foreign_keys: dict
    :param df_filtrado: DataFrame a ser verificado
    :type df_filtrado: pd.DataFrame
    :param cursor: Cursor do banco de dados
    :type cursor: pyodbc.Cursor
    :param esquema: Esquema do banco de dados
    :type esquema: str
    :return: None
    :rtype: None
    """
    print("🔍 Verificando e mapeando chaves estrangeiras...")

    for fk_coluna, (
        ref_tabela,
        ref_coluna,
        ref_coluna_exibicao,
    ) in foreign_keys.items():

        if ref_coluna_exibicao not in df_filtrado.columns:
            raise ValueError(
                f"❌ Erro: coluna '{ref_coluna_exibicao}' não existe no DataFrame!"
            )

        # Buscar valores válidos
        query = f"""
            SELECT DISTINCT {ref_coluna}, {ref_coluna_exibicao}
            FROM {esquema}.{ref_tabela}
            WHERE {ref_coluna_exibicao} IS NOT NULL
        """
        cursor.execute(query)
        valores_validos = cursor.fetchall()

        if not valores_validos:
            raise ValueError(
                f"❌ A tabela {esquema}.{ref_tabela} não possui valores válidos para FK."
            )

        mapa_validos = {str(row[1]).strip().lower(): row[0] for row in valores_validos}

        # Mapeamento
        df_filtrado[fk_coluna] = (
            df_filtrado[ref_coluna_exibicao]
            .astype(str)
            .str.strip()
            .str.lower()
            .map(mapa_validos)
        )

        # Verifica valores sem correspondência
        ausentes = df_filtrado[df_filtrado[fk_coluna].isna()][
            ref_coluna_exibicao
        ].unique()
        if len(ausentes) > 0:
            print("\n⚠️ Valores sem correspondência na FK:")
            for v in ausentes[:20]:
                print("   -", v)
            raise ValueError(
                f"❌ Existem valores sem correspondência para a FK '{fk_coluna}'. Corrija antes de inserir."
            )

        df_filtrado[fk_coluna] = df_filtrado[fk_coluna].astype(int)
        df_filtrado.drop(columns=[ref_coluna_exibicao], inplace=True)

In [ ]:
def inserir_lotes_dados(
    conn: pyodbc.Connection,
    cursor: pyodbc.Cursor,
    tabela: str,
    df_filtrado: pd.DataFrame,
    sql_query: str,
    n_lotes: int,
    total_rows: int,
    original_autocommit: bool,
    esquema: str = "inep",
    usar_identity_insert: bool = False,
    tamanho_bloco: int = 1000,
    max_tentativas: int = 3,
) -> None:
    """
    Insere dados em grandes lotes de dados na tabela com tentativas e tratamento de erros.

    :param conn: Conexão com o banco de dados
    :type conn: pyodbc.Connection
    :param cursor: Cursor do banco de dados
    :type cursor: pyodbc.Cursor
    :param tabela: Nome da tabela de destino
    :type tabela: str
    :param df_filtrado: DataFrame com os dados a serem inseridos
    :type df_filtrado: pd.DataFrame
    :param sql: Comando SQL para inserção
    :type sql: str
    :param n_batches: Número de lotes para inserção
    :type n_batches: int
    :param total_rows: Total de linhas a serem inseridas
    :type total_rows: int
    :param original_autocommit: Estado original do autocommit da conexão
    :type original_autocommit: bool
    :param schema: Esquema do banco de dados
    :type schema: str
    :param usar_identity_insert: Habilitar IDENTITY_INSERT se necessário
    :type usar_identity_insert: bool
    :param tamanho_bloco: Número de linhas por batch
    :type tamanho_bloco: int
    :param max_tentativas: Número máximo de tentativas em caso de falha
    :type max_tentativas: int
    """
    print(f"🚀 Iniciando inserção na tabela {esquema}.{tabela}")
    try:
        conn.autocommit = True
        bar = tqdm(
            total=n_lotes,
            desc=f"Inserindo {tabela}",
            colour="blue",
            ncols=100,
            unit="batch",
            unit_scale=True,
        )
        for batch_i in range(n_lotes):
            inicio = batch_i * tamanho_bloco
            fim = min(inicio + tamanho_bloco, total_rows)
            batch_df = df_filtrado.iloc[inicio:fim]
            data = [tuple(x) for x in batch_df.to_numpy()]

            attempt = 0
            while attempt < max_tentativas:
                try:
                    cursor.fast_executemany = True
                    cursor.executemany(sql_query, data)
                    bar.update(1)
                    break

                except pyodbc.OperationalError as e:
                    attempt += 1
                    tempo_espera = 2**attempt
                    print(
                        f"⚠️ Falha no batch {batch_i+1}: {e}. Nova tentativa em {tempo_espera}s..."
                    )
                    time.sleep(tempo_espera)
                    cursor.close()
                    cursor = conn.cursor()

                except pyodbc.Error as ex:
                    attempt += 1
                    print(f"SQL Server Error: {ex.args[0]} - {ex}")
                    conn.rollback()

                except Exception as e:
                    print(f"❌ Erro irreversível no batch {batch_i+1}: {e}")
                    raise

    finally:
        if usar_identity_insert:
            try:
                cursor.execute(f"SET IDENTITY_INSERT {esquema}.{tabela} OFF;")
            except:
                pass

        conn.autocommit = original_autocommit
        cursor.close()

In [ ]:
def insert_dados(
    conn: pyodbc.Connection,
    tabela: str,
    df_filtrado: pd.DataFrame,
    chunk_size: int = 1000,
    schema: str = "inep",
    foreign_keys: dict = {},
    usar_identity_insert: bool = False,
) -> None:
    """
    Insere o DataFrame em batches usando fast_executemany, com retry e autocommit temporário.
    Inclui tratamento seguro de FOREIGN KEY antes da inserção.

    foreign_keys:
        Exemplo:
        {
            'id_municipio': ('municipio', 'id_municipio', 'nome_municipio')
        }

    :param conn: Conexão com o banco de dados
    :type conn: pyodbc.Connection
    :param tabela: Nome da tabela de destino
    :type tabela: str
    :param df_filtrado: DataFrame a ser inserido
    :type df_filtrado: pd.DataFrame
    :param foreign_keys: Dicionário de chaves estrangeiras para tratamento
    :type foreign_keys: dict
    :param chunk_size: Número de linhas por batch
    :param chunk_size: Tamanho do lote para inserção
    :param schema: Nome do esquema do banco de dados
    :type schema: str
    :param usar_identity_insert: Habilitar IDENTITY_INSERT se necessário
    :type usar_identity_insert: bool
    :return: None
    :rtype: None
    """
    original_autocommit = conn.autocommit
    cursor = conn.cursor()

    # tratar chaves estrangeiras
    if foreign_keys:
        tratar_chaves_estrangeiras(foreign_keys, df_filtrado, cursor, schema)

    # Habilitar IDENTITY_INSERT se necessário
    if usar_identity_insert:
        try:
            cursor.execute(f"SET IDENTITY_INSERT {schema}.{tabela} ON;")
        except Exception as e:
            print(f"⚠️ Não foi possível habilitar IDENTITY_INSERT: {e}")

    # Montar comando SQL de inserção
    colunas = ", ".join(df_filtrado.columns)
    espaços_reservados = ",".join(["?"] * len(df_filtrado.columns))
    sql_query = (
        f"INSERT INTO {schema}.{tabela} ({colunas}) VALUES ({espaços_reservados})"
    )

    total_linhas = len(df_filtrado)
    if total_linhas == 0:
        print("Nenhuma linha para inserir.")
        return

    n_lotes = math.ceil(total_linhas / chunk_size)

    # Inserção dos lotes com tentativas
    inserir_lotes_dados(
        conn,
        cursor,
        tabela,
        df_filtrado,
        sql_query,
        n_lotes,
        total_linhas,
        original_autocommit,
    )

    print(f"✅ Inserção finalizada com sucesso na tabela {schema}.{tabela}.")

In [ ]:
datasets = [
    "data/microdados_censo_da_educacao_superior_2020/dados/MICRODADOS_CADASTRO_IES_2020.CSV",
    "data/microdados_censo_da_educacao_superior_2021/dados/MICRODADOS_CADASTRO_IES_2021.CSV",
    "data/microdados_censo_da_educacao_superior_2022/dados/MICRODADOS_ED_SUP_IES_2022.CSV",
    "data/microdados_censo_da_educacao_superior_2023/dados/MICRODADOS_ED_SUP_IES_2023.CSV",
    "data/microdados_censo_da_educacao_superior_2024/dados/MICRODADOS_ED_SUP_IES_2024.CSV",
]
df_ies_2024 = carregar_dados_superior_ies(
    "data/microdados_censo_da_educacao_superior_2024/dados/MICRODADOS_ED_SUP_IES_2024.CSV"
)

# Tabela Região


In [ ]:
df = (
    df_ies_2024[
        [
            "CO_REGIAO_IES",
            "NO_REGIAO_IES",
        ]
    ]
    .drop_duplicates("CO_REGIAO_IES")
    .reset_index(drop=True)
)
df["CO_REGIAO_IES"] = df["CO_REGIAO_IES"].astype(int)
df.sort_values("CO_REGIAO_IES", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_REGIAO_IES": "id_regiao",
        "NO_REGIAO_IES": "nome_regiao",
    }
)
insert_dados(conn, "UF", df)

# Tabela Municipio


In [ ]:
df = (
    df_ies_2024[["CO_MUNICIPIO_IES", "NO_MUNICIPIO_IES", "CO_UF_IES"]]
    .drop_duplicates("CO_MUNICIPIO_IES")
    .reset_index(drop=True)
)
df["CO_MUNICIPIO_IES"] = df["CO_MUNICIPIO_IES"].astype(int)
df.sort_values("CO_MUNICIPIO_IES", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_MUNICIPIO_IES": "id_municipio",
        "NO_MUNICIPIO_IES": "nome_municipio",
        "CO_UF_IES": "id_uf",
    }
)
insert_dados(conn, "Municipio", df)

# Tabela Endereço


In [ ]:
df = df_ies_2024[
    [
        "DS_ENDERECO_IES",
        "DS_NUMERO_ENDERECO_IES",
        "DS_COMPLEMENTO_ENDERECO_IES",
        "NO_BAIRRO_IES",
        "NU_CEP_IES",
        "CO_MUNICIPIO_IES",
    ]
].reset_index(drop=True)
df["CO_MUNICIPIO_IES"] = df["CO_MUNICIPIO_IES"].astype(int)
df["DS_ENDERECO_IES"] = df["DS_ENDERECO_IES"].str.title()
df["DS_COMPLEMENTO_ENDERECO_IES"] = df["DS_COMPLEMENTO_ENDERECO_IES"].str.title()
df["NO_BAIRRO_IES"] = df["NO_BAIRRO_IES"].str.title()
df.sort_values("CO_MUNICIPIO_IES", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "DS_ENDERECO_IES": "endereco",
        "DS_NUMERO_ENDERECO_IES": "numero_endereco",
        "DS_COMPLEMENTO_ENDERECO_IES": "complemento",
        "NO_BAIRRO_IES": "bairro",
        "NU_CEP_IES": "cep",
        "CO_MUNICIPIO_IES": "id_municipio",
    }
)
insert_dados(conn, "Endereco", df)

# Tabela Mantenedora


In [ ]:
df = (
    df_ies_2024[
        [
            "CO_MANTENEDORA",
            "NO_MANTENEDORA",
            "TP_CATEGORIA_ADMINISTRATIVA",
            "IN_COMUNITARIA",
            "IN_CONFESSIONAL",
        ]
    ]
    .drop_duplicates("CO_MANTENEDORA")
    .reset_index(drop=True)
)
df["CO_MANTENEDORA"] = df["CO_MANTENEDORA"].astype(int)
df["IN_COMUNITARIA"] = df["IN_COMUNITARIA"].astype(int)
df["IN_CONFESSIONAL"] = df["IN_CONFESSIONAL"].astype(int)
df["NO_MANTENEDORA"] = df["NO_MANTENEDORA"].str.title()
df.sort_values("CO_MANTENEDORA", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_MANTENEDORA": "id_mantenedora",
        "NO_MANTENEDORA": "nome_mantenedora",
        "TP_CATEGORIA_ADMINISTRATIVA": "categoria_administrativa",
        "IN_COMUNITARIA": "natureza_juridica_comunitaria",
        "IN_CONFESSIONAL": "natureza_juridica_confessional",
    }
)
insert_dados(conn, "Mantenedora", df)

# Tabela IES (Instituições de Ensino Superior)


In [ ]:
df = (
    df_ies_2024[
        [
            "CO_IES",
            "NO_IES",
            "SG_IES",
            "TP_ORGANIZACAO_ACADEMICA",
            "TP_REDE",
            "IN_CAPITAL_IES",
            "IN_ACESSO_PORTAL_CAPES",
            "IN_ACESSO_OUTRAS_BASES",
            "IN_ASSINA_OUTRA_BASE",
            "IN_REPOSITORIO_INSTITUCIONAL",
            "IN_BUSCA_INTEGRADA",
            "IN_SERVICO_INTERNET",
            "IN_PARTICIPA_REDE_SOCIAL",
            "IN_CATALOGO_ONLINE",
            "CO_MANTENEDORA",
            "DS_ENDERECO_IES",
        ]
    ]
    .drop_duplicates("CO_IES")
    .reset_index(drop=True)
)
df["CO_IES"] = df["CO_IES"].astype(int)
df["IN_CAPITAL_IES"] = df["IN_CAPITAL_IES"].astype(int)
df["CO_MANTENEDORA"] = df["CO_MANTENEDORA"].astype(int)
df["NO_IES"] = df["NO_IES"].str.title()
df.sort_values("CO_IES", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_IES": "id_ies",
        "NO_IES": "nome_ies",
        "SG_IES": "sigla_ies",
        "TP_ORGANIZACAO_ACADEMICA": "organizacao_academica",
        "TP_REDE": "rede_ensino",
        "IN_CAPITAL_IES": "sede_capital",
        "IN_ACESSO_PORTAL_CAPES": "acesso_portal_capes",
        "IN_ACESSO_OUTRAS_BASES": "acesso_outras_bases",
        "IN_ASSINA_OUTRA_BASE": "assina_outra_base",
        "IN_REPOSITORIO_INSTITUCIONAL": "repositorio_institucional",
        "IN_BUSCA_INTEGRADA": "busca_integrada",
        "IN_SERVICO_INTERNET": "servico_internet",
        "IN_PARTICIPA_REDE_SOCIAL": "participa_rede_social",
        "IN_CATALOGO_ONLINE": "catalogo_online",
        "CO_MANTENEDORA": "id_mantenedora",
        "DS_ENDERECO_IES": "endereco",
    }
)
insert_dados(
    conn,
    "IES",
    df,
    foreign_keys={"id_endereco": ("Endereco", "id_endereco", "endereco")},
)

----

In [ ]:
df_curso_2024 = carregar_dados_superior_curso(
    "data/microdados_censo_da_educacao_superior_2024/dados/MICRODADOS_CADASTRO_CURSOS_2024.CSV"
)
df_curso_2024.head()

# Tabela Cine Área Geral

In [ ]:
df = (
    df_curso_2024[
        [
            "CO_CINE_AREA_GERAL",
            "NO_CINE_AREA_GERAL",
        ]
    ]
    .drop_duplicates("CO_CINE_AREA_GERAL")
    .reset_index(drop=True)
)
df["CO_CINE_AREA_GERAL"] = df["CO_CINE_AREA_GERAL"].astype(int)
df.sort_values("CO_CINE_AREA_GERAL", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_CINE_AREA_GERAL": "id_cine_area_geral",
        "NO_CINE_AREA_GERAL": "nome_area_geral",
    }
)
insert_dados(conn, "Cine_Area_Geral", df)

# Tabela Cine Área Específica

In [ ]:
df = (
    df_curso_2024[
        ["CO_CINE_AREA_ESPECIFICA", "NO_CINE_AREA_ESPECIFICA", "CO_CINE_AREA_GERAL"]
    ]
    .drop_duplicates("CO_CINE_AREA_ESPECIFICA")
    .reset_index(drop=True)
)
df["CO_CINE_AREA_ESPECIFICA"] = df["CO_CINE_AREA_ESPECIFICA"].astype(int)
df["CO_CINE_AREA_GERAL"] = df["CO_CINE_AREA_GERAL"].astype(int)
df.sort_values("CO_CINE_AREA_ESPECIFICA", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_CINE_AREA_ESPECIFICA": "id_cine_area_especifica",
        "NO_CINE_AREA_ESPECIFICA": "nome_area_especifica",
        "CO_CINE_AREA_GERAL": "id_cine_area_geral",
    }
)
insert_dados(conn, "Cine_Area_Especifica", df)

# Tabela Cine Área Detalhada

In [ ]:
df = (
    df_curso_2024[
        ["CO_CINE_AREA_DETALHADA", "NO_CINE_AREA_DETALHADA", "CO_CINE_AREA_ESPECIFICA"]
    ]
    .drop_duplicates("CO_CINE_AREA_DETALHADA")
    .reset_index(drop=True)
)
df["CO_CINE_AREA_DETALHADA"] = df["CO_CINE_AREA_DETALHADA"].astype(int)
df["CO_CINE_AREA_ESPECIFICA"] = df["CO_CINE_AREA_ESPECIFICA"].astype(int)
df.sort_values("CO_CINE_AREA_DETALHADA", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_CINE_AREA_DETALHADA": "id_cine_area_detalhada",
        "NO_CINE_AREA_DETALHADA": "nome_area_detalhada",
        "CO_CINE_AREA_ESPECIFICA": "id_cine_area_especifica",
    }
)
insert_dados(conn, "Cine_Area_Detalhada", df)

# Tabela Curso

In [ ]:
df = (
    df_curso_2024[
        [
            "CO_CURSO",
            "NO_CURSO",
            "TP_GRAU_ACADEMICO",
            "IN_GRATUITO",
            "TP_MODALIDADE_ENSINO",
            "TP_NIVEL_ACADEMICO",
            "TP_DIMENSAO",
            "CO_IES",
            "CO_CINE_AREA_DETALHADA",
        ]
    ]
    .drop_duplicates("CO_CURSO")
    .reset_index(drop=True)
)
df["CO_CURSO"] = df["CO_CURSO"].astype(int)
df["CO_IES"] = df["CO_IES"].astype(int)
df["CO_CINE_AREA_DETALHADA"] = df["CO_CINE_AREA_DETALHADA"].astype(int)
df["IN_GRATUITO"] = df["IN_GRATUITO"].astype(int)
df.sort_values("CO_CURSO", inplace=True, ignore_index=True)
df = df.rename(
    columns={
        "CO_CURSO": "id_curso",
        "NO_CURSO": "nome_curso",
        "TP_GRAU_ACADEMICO": "grau_academico",
        "IN_GRATUITO": "gratuito",
        "TP_MODALIDADE_ENSINO": "modalidade_ensino",
        "TP_NIVEL_ACADEMICO": "nivel_academico",
        "TP_DIMENSAO": "oferta",
        "CO_IES": "id_ies",
        "CO_CINE_AREA_DETALHADA": "id_cine_area_detalhada",
    }
)
insert_dados(conn, "Curso", df)

# Tabela IES por Ano

In [ ]:
dfs = []
for dataset in datasets:
    df_ies = carregar_dados_superior_ies(dataset)
    df_ies = df_ies[df_ies["CO_IES"].astype(int).isin(df_ies_2024)]
    df = df_ies[
        ["QT_PERIODICO_ELETRONICO", "QT_LIVRO_ELETRONICO", "CO_IES", "NU_ANO_CENSO"]
    ]
    df["CO_IES"] = df["CO_IES"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_IES", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    df = df.rename(
        columns={
            "QT_PERIODICO_ELETRONICO": "qt_periodico_eletronico",
            "QT_LIVRO_ELETRONICO": "qt_livro_eletronico",
            "CO_IES": "id_ies",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    dfs.append(df)
df_concat = pd.concat(dfs)
df_concat.drop_duplicates(subset=["id_ies", "id_ano"], inplace=True)
insert_dados(conn, "IES_Ano", df_concat)

# Tabela Tecnico Administrativo por Ano

In [ ]:
dfs = []
for dataset in datasets:
    df_ies = carregar_dados_superior_ies(dataset)
    df_ies = df_ies[df_ies["CO_IES"].astype(int).isin(df_ies_2024)]
    df = df_ies[
        [
            "QT_TEC_TOTAL",
            "QT_TEC_FUNDAMENTAL_INCOMP_FEM",
            "QT_TEC_FUNDAMENTAL_INCOMP_MASC",
            "QT_TEC_FUNDAMENTAL_COMP_FEM",
            "QT_TEC_FUNDAMENTAL_COMP_MASC",
            "QT_TEC_MEDIO_FEM",
            "QT_TEC_MEDIO_MASC",
            "QT_TEC_SUPERIOR_FEM",
            "QT_TEC_SUPERIOR_MASC",
            "QT_TEC_ESPECIALIZACAO_FEM",
            "QT_TEC_ESPECIALIZACAO_MASC",
            "QT_TEC_MESTRADO_FEM",
            "QT_TEC_MESTRADO_MASC",
            "QT_TEC_DOUTORADO_FEM",
            "QT_TEC_DOUTORADO_MASC",
            "CO_IES",
            "NU_ANO_CENSO",
        ]
    ]
    df["CO_IES"] = df["CO_IES"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_IES", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    df = df.rename(
        columns={
            "QT_TEC_TOTAL": "qt_tec_total",
            "QT_TEC_FUNDAMENTAL_INCOMP_FEM": "qt_tec_fundamental_incomp_fem",
            "QT_TEC_FUNDAMENTAL_INCOMP_MASC": "qt_tec_fundamental_incomp_masc",
            "QT_TEC_FUNDAMENTAL_COMP_FEM": "qt_tec_fundamental_comp_fem",
            "QT_TEC_FUNDAMENTAL_COMP_MASC": "qt_tec_fundamental_comp_masc",
            "QT_TEC_MEDIO_FEM": "qt_tec_medio_fem",
            "QT_TEC_MEDIO_MASC": "qt_tec_medio_masc",
            "QT_TEC_SUPERIOR_FEM": "qt_tec_superior_fem",
            "QT_TEC_SUPERIOR_MASC": "qt_tec_superior_masc",
            "QT_TEC_ESPECIALIZACAO_FEM": "qt_tec_especializacao_fem",
            "QT_TEC_ESPECIALIZACAO_MASC": "qt_tec_especializacao_masc",
            "QT_TEC_MESTRADO_FEM": "qt_tec_mestrado_fem",
            "QT_TEC_MESTRADO_MASC": "qt_tec_mestrado_masc",
            "QT_TEC_DOUTORADO_FEM": "qt_tec_doutorado_fem",
            "QT_TEC_DOUTORADO_MASC": "qt_tec_doutorado_masc",
            "CO_IES": "id_ies",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    dfs.append(df)
df_concat = pd.concat(dfs)
df_concat.drop_duplicates(subset=["id_ies", "id_ano"], inplace=True)
insert_dados(conn, "Tecnico_Adm_Ano", df_concat)

# Tabela Docente por Ano

In [ ]:
dfs = []
for dataset in datasets:
    df_ies = carregar_dados_superior_ies(dataset)
    df_ies = df_ies[df_ies["CO_IES"].astype(int).isin(df_ies_2024)]
    df = df_ies[
        [
            "QT_DOC_TOTAL",
            "QT_DOC_EXE",
            "QT_DOC_EX_FEMI",
            "QT_DOC_EX_MASC",
            "QT_DOC_EX_SEM_GRAD",
            "QT_DOC_EX_GRAD",
            "QT_DOC_EX_ESP",
            "QT_DOC_EX_MEST",
            "QT_DOC_EX_DOUT",
            "QT_DOC_EX_INT",
            "QT_DOC_EX_INT_DE",
            "QT_DOC_EX_INT_SEM_DE",
            "QT_DOC_EX_PARC",
            "QT_DOC_EX_HOR",
            "QT_DOC_EX_0_29",
            "QT_DOC_EX_30_34",
            "QT_DOC_EX_35_39",
            "QT_DOC_EX_40_44",
            "QT_DOC_EX_45_49",
            "QT_DOC_EX_50_54",
            "QT_DOC_EX_55_59",
            "QT_DOC_EX_60_MAIS",
            "QT_DOC_EX_BRANCA",
            "QT_DOC_EX_PRETA",
            "QT_DOC_EX_PARDA",
            "QT_DOC_EX_AMARELA",
            "QT_DOC_EX_INDIGENA",
            "QT_DOC_EX_COR_ND",
            "QT_DOC_EX_BRA",
            "QT_DOC_EX_EST",
            "QT_DOC_EX_COM_DEFICIENCIA",
            "CO_IES",
            "NU_ANO_CENSO",
        ]
    ]
    df["CO_IES"] = df["CO_IES"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_IES", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    colunas = df.columns.tolist()
    colunas = [col.lower() for col in colunas if col not in ["CO_IES", "NU_ANO_CENSO"]]
    df = df.rename(
        columns={
            col: col.lower()
            for col in df.columns
            if col not in ["CO_IES", "NU_ANO_CENSO"]
        }
    )
    df = df.rename(
        columns={
            "CO_IES": "id_ies",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    dfs.append(df)
df_concat = pd.concat(dfs)
df_concat.drop_duplicates(subset=["id_ies", "id_ano"], inplace=True)
insert_dados(conn, "Docente_Ano", df_concat)

# Tabela Curso por Ano

In [ ]:
for dataset in datasets:
    print(dataset)
    df_curso = carregar_dados_superior_curso(dataset)
    df_curso = df_curso[df_curso["CO_CURSO"].astype(int).isin(df_curso_2024)]
    df = df_curso[
        [
            "QT_CURSO",
            "QT_VG_TOTAL",
            "QT_VG_TOTAL_DIURNO",
            "QT_VG_TOTAL_NOTURNO",
            "QT_VG_TOTAL_EAD",
            "QT_VG_NOVA",
            "QT_VG_PROC_SELETIVO",
            "QT_VG_REMANESC",
            "QT_VG_PROG_ESPECIAL",
            "QT_INSCRITO_TOTAL",
            "QT_INSCRITO_TOTAL_DIURNO",
            "QT_INSCRITO_TOTAL_NOTURNO",
            "QT_INSCRITO_TOTAL_EAD",
            "QT_INSC_VG_NOVA",
            "QT_INSC_PROC_SELETIVO",
            "QT_INSC_VG_REMANESC",
            "QT_INSC_VG_PROG_ESPECIAL",
            "QT_ALUNO_DEFICIENTE",
            "CO_CURSO",
            "NU_ANO_CENSO",
        ]
    ]
    del df_curso
    df["CO_CURSO"] = df["CO_CURSO"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_CURSO", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    colunas = df.columns.tolist()
    colunas = [
        col.lower() for col in colunas if col not in ["CO_CURSO", "NU_ANO_CENSO"]
    ]
    df = df.rename(
        columns={
            col: col.lower()
            for col in df.columns
            if col not in ["CO_CURSO", "NU_ANO_CENSO"]
        }
    )
    df = df.rename(
        columns={
            "CO_CURSO": "id_curso",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    df.drop_duplicates(subset=["id_curso", "id_ano"], inplace=True)
    insert_dados(conn, "Curso_Ano", df)
    del df

# Tabela Ingressante por Ano

In [ ]:
for dataset in datasets:
    print(dataset)
    df_curso = carregar_dados_superior_curso(dataset)
    df_curso = df_curso[df_curso["CO_CURSO"].astype(int).isin(df_curso_2024)]
    df = df_curso[
        [
            "QT_ING",
            "QT_ING_FEM",
            "QT_ING_MASC",
            "QT_ING_DIURNO",
            "QT_ING_NOTURNO",
            "QT_ING_VG_NOVA",
            "QT_ING_VESTIBULAR",
            "QT_ING_ENEM",
            "QT_ING_AVALIACAO_SERIADA",
            "QT_ING_SELECAO_SIMPLIFICA",
            "QT_ING_EGR",
            "QT_ING_OUTRO_TIPO_SELECAO",
            "QT_ING_PROC_SELETIVO",
            "QT_ING_VG_REMANESC",
            "QT_ING_VG_PROG_ESPECIAL",
            "QT_ING_OUTRA_FORMA",
            "QT_ING_0_17",
            "QT_ING_18_24",
            "QT_ING_25_29",
            "QT_ING_30_34",
            "QT_ING_35_39",
            "QT_ING_40_49",
            "QT_ING_50_59",
            "QT_ING_60_MAIS",
            "QT_ING_BRANCA",
            "QT_ING_PRETA",
            "QT_ING_PARDA",
            "QT_ING_AMARELA",
            "QT_ING_INDIGENA",
            "QT_ING_CORND",
            "QT_ING_NACBRAS",
            "QT_ING_NACESTRANG",
            "QT_ING_DEFICIENTE",
            "CO_CURSO",
            "NU_ANO_CENSO",
        ]
    ]
    del df_curso
    df["CO_CURSO"] = df["CO_CURSO"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_CURSO", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    colunas = df.columns.tolist()
    colunas = [
        col.lower() for col in colunas if col not in ["CO_CURSO", "NU_ANO_CENSO"]
    ]
    df = df.rename(
        columns={
            col: col.lower()
            for col in df.columns
            if col not in ["CO_CURSO", "NU_ANO_CENSO"]
        }
    )
    df = df.rename(
        columns={
            "CO_CURSO": "id_curso",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    df.drop_duplicates(subset=["id_curso", "id_ano"], inplace=True)
    insert_dados(conn, "Ingressante_Ano", df)
    del df

# Tabela Matricula por Ano

In [ ]:
for dataset in datasets:
    print(dataset)
    df_curso = carregar_dados_superior_curso(dataset)
    df_curso = df_curso[df_curso["CO_CURSO"].astype(int).isin(df_curso_2024)]
    df = df_curso[
        [
            "QT_MAT",
            "QT_MAT_FEM",
            "QT_MAT_MASC",
            "QT_MAT_DIURNO",
            "QT_MAT_NOTURNO",
            "QT_MAT_0_17",
            "QT_MAT_18_24",
            "QT_MAT_25_29",
            "QT_MAT_30_34",
            "QT_MAT_35_39",
            "QT_MAT_40_49",
            "QT_MAT_50_59",
            "QT_MAT_60_MAIS",
            "QT_MAT_BRANCA",
            "QT_MAT_PRETA",
            "QT_MAT_PARDA",
            "QT_MAT_AMARELA",
            "QT_MAT_INDIGENA",
            "QT_MAT_CORND",
            "CO_CURSO",
            "NU_ANO_CENSO",
        ]
    ]
    del df_curso
    df["CO_CURSO"] = df["CO_CURSO"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_CURSO", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    colunas = df.columns.tolist()
    colunas = [
        col.lower() for col in colunas if col not in ["CO_CURSO", "NU_ANO_CENSO"]
    ]
    df = df.rename(
        columns={
            col: col.lower()
            for col in df.columns
            if col not in ["CO_CURSO", "NU_ANO_CENSO"]
        }
    )
    df = df.rename(
        columns={
            "CO_CURSO": "id_curso",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    df.drop_duplicates(subset=["id_curso", "id_ano"], inplace=True)
    insert_dados(conn, "Matricula_Ano", df)
    del df

# Tabela Concluinte por Ano

In [ ]:
for dataset in datasets:
    print(dataset)
    df_curso = carregar_dados_superior_curso(dataset)
    df_curso = df_curso[df_curso["CO_CURSO"].astype(int).isin(df_curso_2024)]
    df = df_curso[
        [
            "QT_CONC",
            "QT_CONC_FEM",
            "QT_CONC_MASC",
            "QT_CONC_DIURNO",
            "QT_CONC_NOTURNO",
            "QT_CONC_0_17",
            "QT_CONC_18_24",
            "QT_CONC_25_29",
            "QT_CONC_30_34",
            "QT_CONC_35_39",
            "QT_CONC_40_49",
            "QT_CONC_50_59",
            "QT_CONC_60_MAIS",
            "QT_CONC_BRANCA",
            "QT_CONC_PRETA",
            "QT_CONC_PARDA",
            "QT_CONC_AMARELA",
            "QT_CONC_INDIGENA",
            "QT_CONC_CORND",
            "QT_CONC_NACBRAS",
            "QT_CONC_NACESTRANG",
            "QT_CONC_DEFICIENTE",
            "CO_CURSO",
            "NU_ANO_CENSO",
        ]
    ]
    del df_curso
    df["CO_CURSO"] = df["CO_CURSO"].astype(int)
    df["NU_ANO_CENSO"] = df["NU_ANO_CENSO"].astype(int)
    df.sort_values(["CO_CURSO", "NU_ANO_CENSO"], inplace=True, ignore_index=True)
    colunas = df.columns.tolist()
    colunas = [
        col.lower() for col in colunas if col not in ["CO_CURSO", "NU_ANO_CENSO"]
    ]
    df = df.rename(
        columns={
            col: col.lower()
            for col in df.columns
            if col not in ["CO_CURSO", "NU_ANO_CENSO"]
        }
    )
    df = df.rename(
        columns={
            "CO_CURSO": "id_curso",
            "NU_ANO_CENSO": "id_ano",
        }
    )
    df.drop_duplicates(subset=["id_curso", "id_ano"], inplace=True)
    insert_dados(conn, "Concluinte_Ano", df)
    del df